In [67]:
pip install -r requirements.txt

In [68]:
import pandas as pd
import re


# Read the Excel file into a DataFrame
df = pd.read_excel('filename.xlsx')

# Display the shape of the DataFrame and the first few rows
print(df.shape)
df.head()

(5164, 14)


,Unnamed: 0,DOI,Full_Text_url,Full_Text_Content,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,0,10.12688/openreseurope.13841.1,https://open-research-europe.ec.europa.eu/exta...,REVIEW\nSafeguarding COVID-19 and cancer manag...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ey are now also being recognized for their ant...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,immunotherapeutic properties. During SARS-CoV-...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,the effects of antiviral immune responses can ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,using chemotherapeutics via the subsequent imm...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
# Concatenate all columns into a new 'text' column
df['text'] = df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

# Display the first few rows of the DataFrame to verify the result
df.head()

,Unnamed: 0,DOI,Full_Text_url,Full_Text_Content,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,text
0,0,10.12688/openreseurope.13841.1,https://open-research-europe.ec.europa.eu/exta...,REVIEW\nSafeguarding COVID-19 and cancer manag...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 10.12688/openreseurope.13841.1 https://open-...
1,ey are now also being recognized for their ant...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ey are now also being recognized for their ant...
2,immunotherapeutic properties. During SARS-CoV-...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,immunotherapeutic properties. During SARS-CoV-...
3,the effects of antiviral immune responses can ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,the effects of antiviral immune responses can ...
4,using chemotherapeutics via the subsequent imm...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,using chemotherapeutics via the subsequent imm...


In [70]:
# Preview the first few rows of the 'text' column
print(df['text'].head())

0    0 10.12688/openreseurope.13841.1 https://open-...
1    ey are now also being recognized for their ant...
2    immunotherapeutic properties. During SARS-CoV-...
3    the effects of antiviral immune responses can ...
4    using chemotherapeutics via the subsequent imm...
Name: text, dtype: object


In [71]:
# Function to clean the text
def cleanse_text(text):
    unwanted_sections = ["abstract", "method", "methods", "references"]
    for section in unwanted_sections:
        pattern = r'(?i)\b{}\b.*?(?=\n\n|\Z)'.format(section)  # Case-insensitive section removal
        text = re.sub(pattern, '', text, flags=re.DOTALL)

    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces/newlines with a single space
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # Remove non-ASCII characters
    text = remove_leading_zeros(text)
    text = text.strip()
    text = re.sub(r'\. \.', '.', text)  # Fix any erroneous spaces between periods

    # Ensure paragraphs are maintained
    text = re.sub(r'\n\s*\n', '\n\n', text)  # Ensure paragraph breaks are preserved

    return text

# Function to remove leading zeros from numbers
def remove_leading_zeros(text):
    pattern = r'\b0+(\d+(\.\d+)?)\b'
    result = re.sub(pattern, r'\1', text)
    return result

print(df['text'].head())

0    0 10.12688/openreseurope.13841.1 https://open-...
1    ey are now also being recognized for their ant...
2    immunotherapeutic properties. During SARS-CoV-...
3    the effects of antiviral immune responses can ...
4    using chemotherapeutics via the subsequent imm...
Name: text, dtype: object


In [72]:
# Apply the cleansing function to the 'text' column
df['text'] = df['text'].apply(cleanse_text)

# Preview the first few rows of the 'text' column after cleaning
print("\nAfter cleaning:")
print(df['text'].head(10)) 


After cleaning:
0    0 10.12688/openreseurope.13841.1 https://open-...
1    ey are now also being recognized for their ant...
2    immunotherapeutic properties. During SARS-CoV-...
3    the effects of antiviral immune responses can ...
4    using chemotherapeutics via the subsequent imm...
5    indeed the internalization of chemotherapeutic...
6    stream and lymphatic system activates the firs...
7    response as well as recruits second line respo...
8    trophils and macrophages) to the infection sit...
9    efficacy of antiviral therapy is dependent on ...
Name: text, dtype: object


In [73]:
def fix_encoding_issues(text):
    try:
        text = text.encode('latin1').decode('utf-8')
    except UnicodeEncodeError:
        pass  # If encoding fails, just return the original text
    return text

In [77]:
# Placeholder function definitions for extracting specific fields
def extract_bold_title(text):
    text = fix_encoding_issues(text)
    
    # Updated regex to capture bold text and extract content inside <b> or <strong> tags
    title_match = re.search(
        r'(?i)(?:Review|Open Access|Original Article|Research Article)?\s*\n*\s*(<(?:b|strong)>.*?</(?:b|strong)>)',
        text
    )
    
    if title_match:
        # Extract the bold text content
        bold_content = title_match.group(1)
        
        # Remove HTML tags to get the clean text inside the bold tags
        title = re.sub(r'<.*?>', '', bold_content).strip()
        
        # Check if the extracted text is title-like (e.g., reasonably long, starts with an uppercase letter)
        if len(title) > 15 and re.match(r'^[A-Z0-9][^\n]{10,100}$', title):
            return title
    
    return None
    
def extract_doi(text):
    # Fix encoding issues
    text = fix_encoding_issues(text)
    
    # Regex pattern to match a DOI
    match = re.search(r'10\.\d{4,9}/[-._;()/:A-Z0-9]+', text, re.IGNORECASE)
    
    if match:
        doi = match.group(0).strip()
        
        # Clean up any trailing or leading non-DOI characters
        doi = re.sub(r'[^a-zA-Z0-9./:-]+$', '', doi)
        
        return doi
    else:
        return ""

def extract_authors(text):
    text = fix_encoding_issues(text)
    # This regex looks for a list of names separated by commas
    match = re.search(r'(?m)^\s*[A-Z][a-z]+(?: [A-Z]\.)?(?:, [A-Z][a-z]+(?: [A-Z]\.)?)*', text)
    return match.group(0).strip() if match else ""
    
def extract_fulltexturl(text):
    text = fix_encoding_issues(text)  # Fix any encoding issues first
    
    # Use regex to search for a line that contains 'FulltextUrl:' followed by a valid URL
    match = re.search(r'https?://[^\s]+', text)
    
    if match:
        url = match.group(0).strip()
        
        # Remove any trailing non-URL characters (e.g., 'List')
        url = re.sub(r'[^\w:/?=&.-]+$', '', url)  # Remove trailing non-URL characters
        
        return url
    else:
        return ""
  



In [79]:
# Extract specific fields from the cleaned 'text' column
df['DOI'] = df['Full_Text_Content'].astype(str).apply(extract_doi)
df['Title'] = df['Full_Text_Content'].astype(str).apply(extract_title)
df['Authors'] = df['Full_Text_Content'].astype(str).apply(extract_authors)
df['FulltextUrl'] = df['Full_Text_Content'].astype(str).apply(extract_fulltexturl)

# Step 4: Assign the remaining text to 'FulltextContent'
def extract_fulltext_content(text):
    # Remove the extracted fields from the text
    text = re.sub(r'^Title:\s*.*$', '', text, flags=re.MULTILINE)
    text = re.sub(r'DOI:\s*10\.\d{4,9}/[-._;()/:A-Z0-9]+\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Authors:\s*.*?(?=\s*(Title|DOI|URL|$))', '', text, flags=re.IGNORECASE)
    text = re.sub(r'FulltextUrl:\s*https?://\S+', '', text)
    return text.strip()


# Apply the function to each row in the column
#df['FulltextContent'] = df['text'].apply(tokenize_and_structure)

# If needed, flatten the list of paragraphs into individual rows
#df = df.explode('FulltextContent').reset_index(drop=True)

# Apply the extraction function to remove the extracted fields from the FulltextContent
df['FulltextContent'] = df['text'].apply(extract_fulltext_content)

# Step 5: Retain only the required columns
#df_final = df[['Title', 'Doi', 'Authors', 'FulltextUrl', 'FulltextContent']]

# Apply the function to extract Title from the 'Full_Text_Content' column
df['Title'] = df['Full_Text_Content'].astype(str).apply(extract_title)

# Extract the first non-null Title
first_title = df['Title'].dropna().head(1).values[0]

# Assign this Title to every row in the 'Title' column
df['Title'] = first_title

# Apply the function to extract DOI from the 'Full_Text_Content' column
df['DOI'] = df['Full_Text_Content'].astype(str).apply(extract_doi)

# Extract the first non-null DOI
first_doi = df['DOI'].dropna().head(1).values[0]

# Assign this DOI to every row in the 'DOI' column
df['DOI'] = first_doi

# Extract the first non-null URL from the 'FulltextUrl' column
first_url = df['FulltextUrl'].dropna().iloc[0]  # or .head(1).values[0]

# Step 2: Assign this URL to every row in the 'FulltextUrl' column
df['FulltextUrl'] = first_url

df_final = df[['Title','DOI','FulltextUrl', 'FulltextContent']]


# Step 6: Save the final cleaned DataFrame to a CSV file
df_final.to_csv('cleaned_data.csv', index=False)

# Optional: Preview the first few rows of the final DataFrame
print("\nFinal DataFrame:")
print(df_final.head(10))


Final DataFrame:
                                               Title  \
0  Safeguarding COVID-19 and cancer management: d...   
1  Safeguarding COVID-19 and cancer management: d...   
2  Safeguarding COVID-19 and cancer management: d...   
3  Safeguarding COVID-19 and cancer management: d...   
4  Safeguarding COVID-19 and cancer management: d...   
5  Safeguarding COVID-19 and cancer management: d...   
6  Safeguarding COVID-19 and cancer management: d...   
7  Safeguarding COVID-19 and cancer management: d...   
8  Safeguarding COVID-19 and cancer management: d...   
9  Safeguarding COVID-19 and cancer management: d...   

                              DOI  \
0  10.12688/openreseurope.13841.1   
1  10.12688/openreseurope.13841.1   
2  10.12688/openreseurope.13841.1   
3  10.12688/openreseurope.13841.1   
4  10.12688/openreseurope.13841.1   
5  10.12688/openreseurope.13841.1   
6  10.12688/openreseurope.13841.1   
7  10.12688/openreseurope.13841.1   
8  10.12688/openreseurope.13841.